In [1]:
import datetime

import time
import os
import sys
from pathlib import Path

import numpy as np
from scipy.stats import lognorm
import pandas as pd

from astropy import stats
from astropy.io import fits
from astropy.coordinates import EarthLocation
from astropy.time import Time
from astropy.utils import iers
import astropy.units as u
import astroplan

import matplotlib
#matplotlib.use('nbagg')
from matplotlib import style
style.use('ggplot')
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [3]:
%cd /home/halcoll/halcoll/halcoll/data

/home/halcoll/halcoll/halcoll/data


In [7]:
from astropy.utils import iers
iers.IERS_A_URL = 'ftp://cddis.gsfc.nasa.gov/pub/products/iers/finals2000A.all'
astroplan.download_IERS_A()

In [50]:
years = []
for y in [2016, 2017, 2018, 2019]:
    years.append(pd.read_csv(f"{y}_e_series_trimmed.csv"))
temp_data = pd.concat(years, sort=True)

In [9]:
lat = "31:41:19.6"
lon = "-110:53:04.4"
elevation = 2600 * u.m
location = EarthLocation.from_geodetic(lon, lat, elevation)
mmt = astroplan.Observer(name="MMTO", location=location, timezone="US/Arizona", pressure=0*u.mbar)
tset = mmt.sun_set_time(Time(datetime.datetime.now()), which='next', horizon=-0.8333*u.deg)
tset.isot

'2019-11-20T00:22:28.313'

In [10]:
tset_r = tset + datetime.timedelta(seconds=30)
tset_r.strftime("%H %M")

'00 22'

In [11]:
from skyfield import api
ts = api.load.timescale()
e = api.load('de421.bsp')
from skyfield import almanac

In [12]:
mmt_sf = api.Topos(latitude_degrees=location.lat.value, longitude_degrees=location.lon.value, elevation_m=2600.0)

In [21]:
t0 = ts.utc(2016, 1, 1, 0)
t1 = ts.utc(2019, 12, 31, 23)
t, y = almanac.find_discrete(t0, t1, almanac.sunrise_sunset(e, mmt_sf))

In [29]:
t_str = np.array(t.utc_iso(), dtype=np.unicode_)
t_dt = np.array(t.utc_datetime())

In [30]:
# y is true when sun is up, false when down. so sunset is when y is False.
sunsets = t_str[y == False]
sunsets_dt = t_dt[y == False]
sunsets

array(['2016-01-01T00:30:04Z', '2016-01-02T00:30:47Z',
       '2016-01-03T00:31:31Z', ..., '2019-12-29T00:28:04Z',
       '2019-12-30T00:28:43Z', '2019-12-31T00:29:24Z'], dtype='<U20')

In [52]:
temp_data = temp_data.set_index(pd.DatetimeIndex(temp_data['ts'], name='mst')).tz_localize('MST').drop(columns=['ts'])

In [53]:
temp_data

,cell_e_series_backplate_C,cell_e_series_chamber_ambient_C,cell_e_series_frontplate_C,cell_e_series_in_front_of_primary_C,cell_e_series_lower_plenum_C,cell_e_series_midplate_C,cell_e_series_outside_ambient_C
mst,,,,,,,
2016-01-01 00:00:00-07:00,1.82905,1.71735,1.7748,1.6990,1.78825,1.8217,1.54470
2016-01-01 00:05:00-07:00,1.82340,1.68600,1.7659,1.6806,1.76570,1.8120,1.53410
2016-01-01 00:10:00-07:00,1.82435,1.70410,1.7618,1.6727,1.75620,1.8087,1.52885
2016-01-01 00:15:00-07:00,1.81050,1.71710,1.7481,1.6543,1.75790,1.7965,1.54970
2016-01-01 00:20:00-07:00,1.81450,1.76420,1.7586,1.6753,1.78190,1.8041,1.61250
...,...,...,...,...,...,...,...
2019-05-28 09:35:00-07:00,3.57370,3.92340,3.5252,3.2669,3.63620,3.5350,4.00680
2019-05-28 09:40:00-07:00,3.57370,3.92340,3.5252,3.2669,3.63620,3.5350,4.00680
2019-05-28 09:45:00-07:00,3.57370,3.92340,3.5252,3.2669,3.63620,3.5350,4.00680


In [67]:
sunsets_df = pd.DataFrame(pd.to_datetime(sunsets).tz_convert("MST"), columns=['sunset'])
sunsets_df = sunsets_df.set_index(pd.DatetimeIndex(sunsets_df['sunset'], name='mst'))
sunsets_df

,sunset
mst,
2015-12-31 17:30:04-07:00,2015-12-31 17:30:04-07:00
2016-01-01 17:30:47-07:00,2016-01-01 17:30:47-07:00
2016-01-02 17:31:31-07:00,2016-01-02 17:31:31-07:00
2016-01-03 17:32:15-07:00,2016-01-03 17:32:15-07:00
2016-01-04 17:33:01-07:00,2016-01-04 17:33:01-07:00
...,...
2019-12-26 17:26:48-07:00,2019-12-26 17:26:48-07:00
2019-12-27 17:27:25-07:00,2019-12-27 17:27:25-07:00
2019-12-28 17:28:04-07:00,2019-12-28 17:28:04-07:00


In [68]:
sunset_temp_data = pd.merge_asof(temp_data, sunsets_df, on='mst')

In [71]:
sunset_temp_data['sunset_diff'] = sunset_temp_data['mst'] - sunset_temp_data['sunset']
sunset_temp_data

,mst,cell_e_series_backplate_C,cell_e_series_chamber_ambient_C,cell_e_series_frontplate_C,cell_e_series_in_front_of_primary_C,cell_e_series_lower_plenum_C,cell_e_series_midplate_C,cell_e_series_outside_ambient_C,sunset,sunset_diff
0,2016-01-01 00:00:00-07:00,1.82905,1.71735,1.7748,1.6990,1.78825,1.8217,1.54470,2015-12-31 17:30:04-07:00,06:29:56
1,2016-01-01 00:05:00-07:00,1.82340,1.68600,1.7659,1.6806,1.76570,1.8120,1.53410,2015-12-31 17:30:04-07:00,06:34:56
2,2016-01-01 00:10:00-07:00,1.82435,1.70410,1.7618,1.6727,1.75620,1.8087,1.52885,2015-12-31 17:30:04-07:00,06:39:56
3,2016-01-01 00:15:00-07:00,1.81050,1.71710,1.7481,1.6543,1.75790,1.7965,1.54970,2015-12-31 17:30:04-07:00,06:44:56
4,2016-01-01 00:20:00-07:00,1.81450,1.76420,1.7586,1.6753,1.78190,1.8041,1.61250,2015-12-31 17:30:04-07:00,06:49:56
...,...,...,...,...,...,...,...,...,...,...
355251,2019-05-28 09:35:00-07:00,3.57370,3.92340,3.5252,3.2669,3.63620,3.5350,4.00680,2019-05-27 19:21:03-07:00,14:13:57
355252,2019-05-28 09:40:00-07:00,3.57370,3.92340,3.5252,3.2669,3.63620,3.5350,4.00680,2019-05-27 19:21:03-07:00,14:18:57
355253,2019-05-28 09:45:00-07:00,3.57370,3.92340,3.5252,3.2669,3.63620,3.5350,4.00680,2019-05-27 19:21:03-07:00,14:23:57
355254,2019-05-28 09:50:00-07:00,3.57370,3.92340,3.5252,3.2669,3.63620,3.5350,4.00680,2019-05-27 19:21:03-07:00,14:28:57


In [77]:
sunset_temp_data[sunset_temp_data['sunset_diff'] > pd.Timedelta('23 hours 55 minutes')]

,mst,cell_e_series_backplate_C,cell_e_series_chamber_ambient_C,cell_e_series_frontplate_C,cell_e_series_in_front_of_primary_C,cell_e_series_lower_plenum_C,cell_e_series_midplate_C,cell_e_series_outside_ambient_C,sunset,sunset_diff
210,2016-01-01 17:30:00-07:00,3.98540,5.25160,3.92100,4.47690,4.32980,4.06810,5.98350,2015-12-31 17:30:04-07:00,0 days 23:59:56
498,2016-01-02 17:30:00-07:00,5.09995,7.16010,5.04715,5.94485,5.42020,5.18635,5.34010,2016-01-01 17:30:47-07:00,0 days 23:59:13
786,2016-01-03 17:30:00-07:00,6.75395,8.73735,6.73895,7.73255,7.15165,6.86290,5.60830,2016-01-02 17:31:31-07:00,0 days 23:58:29
1074,2016-01-04 17:30:00-07:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-01-03 17:32:15-07:00,0 days 23:57:45
1362,2016-01-05 17:30:00-07:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-01-04 17:33:01-07:00,0 days 23:56:59
...,...,...,...,...,...,...,...,...,...,...
354215,2019-05-24 19:15:00-07:00,8.13070,12.20750,8.16110,9.96170,8.03110,8.10130,11.14480,2019-05-23 19:18:33-07:00,0 days 23:56:27
354503,2019-05-25 19:15:00-07:00,8.60975,12.06065,8.69620,11.26490,8.39125,8.53585,11.91960,2019-05-24 19:19:12-07:00,0 days 23:55:48
354791,2019-05-26 19:15:00-07:00,7.41800,9.51495,7.48995,9.50715,7.14975,7.33925,8.48320,2019-05-25 19:19:49-07:00,0 days 23:55:11
354792,2019-05-26 19:20:00-07:00,7.40500,9.09220,7.48430,9.47890,7.15060,7.32650,8.08530,2019-05-25 19:19:49-07:00,1 days 00:00:11
